In [1]:
# ✅ VERIFY INSTALLATION
# Since you have uploaded the .whl to your Fabric Environment, it should be installed automatically.
# Run this cell to confirm the correct version (v0.1.1) is loaded.

import importlib.metadata

try:
    version = importlib.metadata.version("usf_fabric_monitoring")
    print(f"✅ Library found: usf_fabric_monitoring v{version}")
    
    if version == "0.1.1":
        print("   You are using the correct version.")
    else:
        print(f"⚠️  WARNING: Expected v0.1.1 but found v{version}.")
        print("   Please check your Fabric Environment settings and ensure the new wheel is published.")
        
except importlib.metadata.PackageNotFoundError:
    print("❌ Library NOT found.")
    print("   Please ensure you have attached the 'Fabric Environment' containing the .whl file to this notebook.")
    print("   Alternatively, upload the .whl file to the Lakehouse 'Files' section and pip install it from there.")

StatementMeta(, a26be5a9-27af-4736-9715-3a1871b86953, 5, Finished, Available, Finished)

✅ Library found: usf_fabric_monitoring v0.1.1
   You are using the correct version.


# Monitor Hub Analysis Pipeline

## Overview
This notebook executes the **Monitor Hub Analysis Pipeline**, which is designed to provide deep insights into Microsoft Fabric activity. It extracts historical data, calculates key performance metrics, and generates comprehensive reports to help identify:
- Constant failures and reliability issues.
- Excess activity by users, locations, or domains.
- Historical performance trends over the last 90 days.

## New in v0.1.2: Sequential Orchestration
The pipeline now automatically handles the entire data lifecycle in a strict sequence to ensure accuracy:
1. **Activity Extraction**: Fetches high-level activity logs from the Admin API.
2. **Job Detail Extraction**: **(New)** Automatically fetches detailed job execution history (JSON) to capture true failure states.
3. **Data Merging**: Combines Activities and Jobs to ensure "Key Measurables" reflect actual failures, not just submission success.
4. **Analysis**: Generates reports based on the fully enriched dataset.

## How to Use
1. **Install Package**: The first cell installs the `usf_fabric_monitoring` package into the current session.
2. **Configure Credentials**: Ensure your Service Principal credentials (`AZURE_CLIENT_ID`, `AZURE_CLIENT_SECRET`, `AZURE_TENANT_ID`) are available. The notebook attempts to load them from a `.env` file in the Lakehouse or you can configure Azure Key Vault.
3. **Set Parameters**:
    - `DAYS_TO_ANALYZE`: Number of days of history to fetch (default: 90).
    - `OUTPUT_DIR`: Lakehouse path where reports will be saved.
4. **Run Analysis**: Execute the pipeline cell. It will:
    - Fetch data from Fabric APIs.
    - Process and enrich the data.
    - Save CSV reports to the specified `OUTPUT_DIR`.

In [2]:
from usf_fabric_monitoring.core.pipeline import MonitorHubPipeline
import os

StatementMeta(, a26be5a9-27af-4736-9715-3a1871b86953, 6, Finished, Available, Finished)

In [ ]:
import inspect
import usf_fabric_monitoring

print(f"📦 Package Location: {os.path.dirname(usf_fabric_monitoring.__file__)}")

# Verify we are running the NEW code (v0.1.2)
try:
    src = inspect.getsource(MonitorHubPipeline._load_detailed_jobs)
    if "jobs_*.json" in src:
        print("✅ SUCCESS: You are running the updated code (v0.1.2).")
        print("   Feature Verified: Sequential Job Extraction & Merging")
    else:
        print("❌ WARNING: You are still running the OLD code.")
        print("   👉 ACTION: Restart the kernel and run the install cell above again.")
except AttributeError:
    print("❌ WARNING: Method '_load_detailed_jobs' not found. You are running an outdated version.")
except Exception as e:
    print(f"⚠️ Could not verify source code: {e}")

StatementMeta(, a26be5a9-27af-4736-9715-3a1871b86953, 7, Finished, Available, Finished)

📦 Package Location: /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/usf_fabric_monitoring
✅ SUCCESS: You are running the updated code (v0.1.1).
   Output will go to: .../raw_data/ (Persistent Cache)


In [4]:
import os
from dotenv import load_dotenv

# --- CREDENTIAL MANAGEMENT ---

# Option 1: Load from .env file in Lakehouse (Easiest migration)
# Upload your .env file to the 'Files' section of your Lakehouse
ENV_PATH = "/lakehouse/default/Files/dot_env_files/.env"
if os.path.exists(ENV_PATH):
    print(f"Loading configuration from {ENV_PATH}")
    load_dotenv(ENV_PATH)
else:
    print(f"Warning: No .env file found at {ENV_PATH}")

# Option 2: Load from Azure Key Vault (Best Practice)
# Uncomment and configure this section to use Azure Key Vault
# try:
#     from notebookutils import mssparkutils
#     KEY_VAULT_NAME = "YourKeyVaultName"
#     os.environ["AZURE_CLIENT_ID"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Client-ID")
#     os.environ["AZURE_CLIENT_SECRET"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Client-Secret")
#     os.environ["AZURE_TENANT_ID"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Tenant-ID")
# except ImportError:
#     pass # Not running in Fabric or notebookutils not available
# except Exception as e:
#     print(f"Key Vault access failed: {e}")

# Verify credentials are present
required_vars = ["AZURE_CLIENT_ID", "AZURE_CLIENT_SECRET", "AZURE_TENANT_ID"]
missing = [v for v in required_vars if not os.getenv(v)]
if missing:
    print(f"❌ Missing required environment variables: {', '.join(missing)}")
else:
    print("✅ Credentials configured successfully")

StatementMeta(, a26be5a9-27af-4736-9715-3a1871b86953, 8, Finished, Available, Finished)

Loading configuration from /lakehouse/default/Files/dot_env_files/.env
✅ Credentials configured successfully


In [5]:
# Configuration
DAYS_TO_ANALYZE = 28
OUTPUT_DIR = "/lakehouse/default/Files/monitor_hub_analysis" # Adjust for Fabric Lakehouse path if needed
# Or use local path for testing
# OUTPUT_DIR = "exports/monitor_hub_analysis"

StatementMeta(, a26be5a9-27af-4736-9715-3a1871b86953, 9, Finished, Available, Finished)

In [6]:
pipeline = MonitorHubPipeline(OUTPUT_DIR)
results = pipeline.run_complete_analysis(days=DAYS_TO_ANALYZE)
pipeline.print_results_summary(results)

StatementMeta(, a26be5a9-27af-4736-9715-3a1871b86953, 10, Finished, Available, Finished)

2025-12-01 19:10:38 | INFO | usf_fabric_monitoring.core.pipeline | Monitor Hub Pipeline initialized
2025-12-01 19:10:38 | INFO | usf_fabric_monitoring.core.pipeline | Starting Monitor Hub analysis for 28 days (API max 28)
2025-12-01 19:10:38 | INFO | usf_fabric_monitoring.core.pipeline | Step 1: Extracting historical activities from Fabric APIs
   ✅ Fetched 510720 activities total.          
  ✓ 2025-11-04: 44889 activities (Exported)
  ✓ 2025-11-05: 39096 activities (Exported)
  ✓ 2025-11-06: 47610 activities (Exported)
  ✓ 2025-11-07: 47986 activities (Exported)
  ✓ 2025-11-08: 21412 activities (Exported)
  ✓ 2025-11-09: 27379 activities (Exported)
  ✓ 2025-11-10: 47445 activities (Exported)
  ✓ 2025-11-11: 47894 activities (Exported)
  ✓ 2025-11-12: 46173 activities (Exported)
  ✓ 2025-11-13: 49801 activities (Exported)
  ✓ 2025-11-14: 48379 activities (Exported)
  ✓ 2025-11-15: 26140 activities (Exported)
  ✓ 2025-11-16: 33476 activities (Exported)
  ✓ 2025-11-17: 63229 activities 

Failed to fetch items for workspace 00000000-0000-0000-0000-000000000000: 400 Client Error: Bad Request for url: https://api.fabric.microsoft.com/v1/workspaces/00000000-0000-0000-0000-000000000000/items
Failed to fetch items for workspace 895691bd-62c2-4858-b6ea-86e95f88f3c1: 401 Client Error: Unauthorized for url: https://api.fabric.microsoft.com/v1/workspaces/895691bd-62c2-4858-b6ea-86e95f88f3c1/items
Failed to fetch items for workspace 2b01f250-d836-4457-a615-1ff3a4f9121d: 404 Client Error: Not Found for url: https://api.fabric.microsoft.com/v1/workspaces/2b01f250-d836-4457-a615-1ff3a4f9121d/items
Failed to fetch items for workspace 1a59083b-cd4e-4e09-8628-ebdc54e360c0: 401 Client Error: Unauthorized for url: https://api.fabric.microsoft.com/v1/workspaces/1a59083b-cd4e-4e09-8628-ebdc54e360c0/items
Failed to fetch items for workspace 7d6f0b25-8c30-4009-868b-cb0aa1481e47: 401 Client Error: Unauthorized for url: https://api.fabric.microsoft.com/v1/workspaces/7d6f0b25-8c30-4009-868b-cb0